In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# "coderpotter/T5-for-Adversarial-Paraphrasing"
# ok, few artifacts in the data, seemed not terrible, 850MB

#"shrishail/t5_paraphrase_msrp_paws"
# 250MB, seemed a bit weaker

# "ramsrigouthamg/t5_sentence_paraphraser"
# not bad, 850MB, seemed pretty reasonable for most things. 

# "prithivida/parrot_paraphraser_on_T5"
# you know what, i think this is one of the better ones I've seen.  850MB 

# "ceshine/t5-paraphrase-quora-paws"
# honestly not bad but its not as good as the parrot paraphraser. 

# "ramsrigouthamg/t5-large-paraphraser-diverse-high-quality"
# This model is huge - 2.75 GB. 
# But it has the best paraphrases, for sure. 

# Conclusion
# For testing, seeing what's good  - go with "prithivida/parrot_paraphraser_on_T5"
# For when you finally do it, go with "ramsrigouthamg/t5-large-paraphraser-diverse-high-quality"


tokenizer = AutoTokenizer.from_pretrained("prithivida/parrot_paraphraser_on_T5")

model = AutoModelForSeq2SeqLM.from_pretrained("prithivida/parrot_paraphraser_on_T5")

In [ ]:
import torch

In [ ]:
orig_l = [
    "the actors pull out all the stops in nearly every scene , but to diminishing effect . the characters never change .",
    "compassionately explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children .",
    "apart from anything else , this is one of the best-sustained ideas i have ever seen on the screen .",
    "my name is Tom and I like driving cars", 
    "I like this apple", 
]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)



prefix = "paraphrase: "
text = [prefix + sen + " </s>" for sen in orig_l]
encoding = tokenizer(orig_l, padding=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

# outputs = model.generate(input_ids=input_ids, attention_mask=attention_masks, num_beams = 8, num_return_sequences=3, max_length=256,
#                          do_sample=True, top_p=120, temperature=1, return_dict_in_generate=True,
#                                                      output_scores=True)

model_output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_masks,
    do_sample=True,
    max_length=60,
    min_length=5,
    top_k=120,
    top_p=0.98,
 #   early_stopping=True,
    num_return_sequences=4,
)
tokenizer.batch_decode(model_output, skip_special_tokens=True)

# outputs = []
# for output in model_output:
#     generated_sent = tokenizer.decode(
#         output, skip_special_tokens=True, clean_up_tokenization_spaces=True
#     )
#     if (
#         generated_sent.lower() != sentence.lower()
#         and generated_sent not in outputs
#     ):
#         outputs.append(generated_sent)
# return outputs

['the actors pull out almost every scene - for a diminishing effect the characters never change.',
 'actors pull out all the stops in nearly every scene, but to diminishing effect the characters never change.',
 'the actors in nearly every scene pull out everything but to diminishing effect the characters never change from their own.',
 'actors pulled out all stops in nearly every scene, but to diminishing effect. characters never changed.',
 'In compassionately exploring the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children he writ large.',
 'A compassionately explored approach explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children.',
 'compassionately explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children.',
 'compassionately examines the seemingly irreconcilable s

In [ ]:
model_output

tensor([[    0,     8, 10485,  3197,    91,   966,   334,  3112,     3,    18,
            21,     3,     9, 26999,    53,  1504,     8,  2850,   470,   483,
             3,     5,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0, 10485,  3197,    91,    66,     8, 10796,    16,  2111,   334,
          3112,     3,     6,    68,    12, 26999,    53,  1504,     8,  2850,
           470,   483,     3,     5,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0,     8, 10485,    16,  2111,   334,  3112,  3197,    91,   762,
            68,    12, 26999,    53,  1504,     8,  2850,   470,   483,    45,
            70,   293,     3,     5,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0, 10485,  6756,    91,    66, 10796

In [ ]:
tokenizer.pad_token_id

0

In [ ]:
def get_paraphrases(sentence, prefix="paraphrase: ", n_predictions=5, top_k=120, max_length=256):
        text = prefix + sentence + " </s>"
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        encoding = tokenizer.encode_plus(
            text, pad_to_max_length=True, return_tensors="pt"
        )
        input_ids, attention_masks = encoding["input_ids"].to(device), encoding[
            "attention_mask"
        ].to(device)

        model_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_masks,
            do_sample=True,
            max_length=max_length,
            top_k=top_k,
            top_p=0.98,
            early_stopping=True,
            num_return_sequences=n_predictions,
        )

        outputs = []
        for output in model_output:
            generated_sent = tokenizer.decode(
                output, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            if (
                generated_sent.lower() != sentence.lower()
                and generated_sent not in outputs
            ):
                outputs.append(generated_sent)
        return outputs

paraphrases = get_paraphrases("The actors pull out all the stops in nearly every scene , but to diminishing effect . the characters never change.")

/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2251: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
paraphrases

['The actors pull out everything in almost every scene but to an infinite effect the characters never change.',
 'In nearly every scene the actors pull out all the stops but to diminishing effects their characters never change.',
 'The actors pull out everything in nearly every scene but with diminishing effect. the characters never change.',
 'The actors pull out all stops in nearly every scene but to diminishing effect. the characters never change.',
 'The actors pull out all the stops in nearly every scene but to a declining effect. The characters never change.']